In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

## 读写文本格式的数据

In [2]:
!type C:\Python3\ex1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [3]:
df = pd.read_csv('ex1.csv')

In [4]:
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


也可以用read_table，只不过需要指定分隔符而已：

In [5]:
pd.read_table('ex1.csv', sep=',')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


并不是所有文件都有标题行：

In [6]:
!type C:\Python3\ex2.csv

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


读入该文件的办法有两个，可以让pandas为其分配默认的列名，也可以自定义列名：

In [7]:
pd.read_csv('ex2.csv')

,1,2,3,4,hello
0,5,6,7,8,world
1,9,10,11,12,foo


In [8]:
pd.read_csv('ex2.csv', header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [9]:
pd.read_csv('ex2.csv', names=['a', 'b', 'c', 'd', 'message'])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


让message列做成DataFrame的索引：

In [10]:
names=['a', 'b', 'c', 'd', 'message']

In [11]:
pd.read_csv('ex2.csv', names=names, index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


如果希望将多个列做成一个层次化索引，只需传入由列编号或列名组成的列表即可：

In [12]:
!type C:\Python3\csv_mindex.csv

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


In [13]:
parsed = pd.read_csv('csv_mindex.csv', index_col=['key1', 'key2'])

In [14]:
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

有些表格可能不是用固定的分隔符去分隔字符的（比如空白符或其他模式）。  
对于这种情况，可以编写一个正则表达式来作为read_table的分隔符。
```pd.read_table(file_path, sep='\s+') # \s表示空白字符
```

用skiprows跳过文件的第一行、第三行和第四行：
```pd.read_csv(file_path, skiprows=[0, 2, 3])```

缺失数据经常要么没有（空字符串），要么用某个标记值表示。  
默认情况下，pandas会用一组经常出现的标记值进行识别，如NA、-1.#IND以及NULL等。  
na_values可以接受一组用于表示缺失值的字符串：
```pd.read_csv(file_path, na_values=['NULL'])```  
可以用一个字典为各列指定不同的NA标记值：  
```
sentinels = {'message': ['foo', 'NA'],
             'something': ['two']}
pd.read_csv(file_path, na_values=sentinels)

```

## 逐块读取文本文件

如果只想读取几行，通过nrows进行指定即可。  
要逐块读取文件，需要设置chunksize（行数）

## 将数据写出到文本格式

In [15]:
!type C:\Python3\ex1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [16]:
data = pd.read_csv('ex1.csv')

In [17]:
data

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


利用DataFrame的to_csv方法，将数据写到csv文件：

In [18]:
data.to_csv('out.csv')

In [19]:
!type C:\python3\out.csv

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [20]:
import sys
data.to_csv(sys.stdout, sep='|')

|a|b|c|d|message
0|1|2|3|4|hello
1|5|6|7|8|world
2|9|10|11|12|foo


缺失值在输出结果中会被表示为空字符串，你可能希望将其表示为别的标记值：  
```data.to_csv(sys.stout, na_rep='NULL')```

如果没有设置其他选项，则会写出行和列的标签。它们也可以被禁用：

In [21]:
data.to_csv(sys.stdout, index=False, header=False)

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


此外，你还可以只写出一部分列，并以你指定的顺序排列：

In [22]:
data.to_csv(sys.stdout, index=False, columns=['a', 'b', 'c'])

a,b,c
1,2,3
5,6,7
9,10,11


Series也有一个to_csv方法：

In [23]:
dates = pd.date_range('1/1/2000', periods=7)

In [24]:
dates

DatetimeIndex(['2000-01-01', '2000-01-02', '2000-01-03', '2000-01-04',
               '2000-01-05', '2000-01-06', '2000-01-07'],
              dtype='datetime64[ns]', freq='D')

In [25]:
ts = Series(np.arange(7), index=dates)

In [26]:
ts

2000-01-01    0
2000-01-02    1
2000-01-03    2
2000-01-04    3
2000-01-05    4
2000-01-06    5
2000-01-07    6
Freq: D, dtype: int32

In [27]:
ts.to_csv('tseries.csv')

In [28]:
!type tseries.csv

2000-01-01,0
2000-01-02,1
2000-01-03,2
2000-01-04,3
2000-01-05,4
2000-01-06,5
2000-01-07,6


In [29]:
Series.from_csv('tseries.csv', parse_dates=True)

C:\Anaconda3\lib\site-packages\pandas\core\series.py:2890: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  infer_datetime_format=infer_datetime_format)


2000-01-01    0
2000-01-02    1
2000-01-03    2
2000-01-04    3
2000-01-05    4
2000-01-06    5
2000-01-07    6
dtype: int64

## 手工处理分隔符格式

大部分存储在磁盘上的表格型数据都能用pandas.read_table进行加载。然而，有时还是需要做一些手工处理。  
由于接收到含有畸形行的文件而使read_table出毛病的情况并不少见。

In [30]:
!type ex7.csv

"a","b","c"
"1","2","3"
"1","2","3","4"


In [31]:
import csv

In [32]:
with open('ex7.csv') as f:
    reader = csv.reader(f)
    for line in reader:
        print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3', '4']


现在，为了使数据格式合乎要求，需要对其做一些整理工作：

In [33]:
lines = list(csv.reader(open('ex7.csv')))

In [34]:
lines

[['a', 'b', 'c'], ['1', '2', '3'], ['1', '2', '3', '4']]

In [35]:
header, values = lines[0], lines[1:]

In [36]:
data_dict = {h: v for h, v in zip(header, zip(*values))}

In [37]:
header

['a', 'b', 'c']

In [38]:
values

[['1', '2', '3'], ['1', '2', '3', '4']]

In [39]:
for x in zip(values):
    print(x)

(['1', '2', '3'],)
(['1', '2', '3', '4'],)


In [40]:
for x in zip(*values):
    print(x)

('1', '1')
('2', '2')
('3', '3')


In [41]:
for y in zip(header, zip(*values)):
    print(y)

('a', ('1', '1'))
('b', ('2', '2'))
('c', ('3', '3'))


In [42]:
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

CSV文件的形式有很多。只需定义csv.Dialect的一个子类即可定义出新格式（如专门的分隔符、字符串引用约定、行结束符等）：

In [43]:
class my_dialect(csv.Dialect):
    lineterminator = '\n'
    delimiter = ';'
    quotechar = '"'
    quoting = 0 # quoting : int or csv.QUOTE_* instance, default 0,控制csv中的引号常量。可选 QUOTE_MINIMAL (0), QUOTE_ALL (1), QUOTE_NONNUMERIC (2) or QUOTE_NONE (3)

In [44]:
reader = csv.reader(open('ex7.csv'), dialect=my_dialect)

各个CSV语支的参数也可以关键字的形式提供给csv.reader，而无需定义子类：

In [45]:
reader = csv.reader(open('ex7.csv'), delimiter='|')

要手工输出分隔符文件，你可以使用csv.writer.

In [46]:
with open('mydata.csv', 'w') as f:
    writer = csv.writer(f, dialect=my_dialect)
    writer.writerow(('one', 'two', 'three'))
    writer.writerow(('1', '2', '3'))
    writer.writerow(('4', '5', '6'))
    writer.writerow(('7', '8', '9'))

In [47]:
!type mydata.csv

one;two;three
1;2;3
4;5;6
7;8;9


## JSON数据
JSON（JavaScript Object Notation的简称）已经成为通过HTTP请求在Web浏览器和其他应用程序之间发送数据的标准格式之一。它是一种比表格型文本格式（如CSV）灵活得多的数据格式。

## XML和HTML：Web信息收集

## 二进制数据格式
实现数据的二进制格式存储最简单的办法之一是使用Python内置的pickle序列化。  
为了使用方便，pandas对象都有一个用于将数据以pickle形式保存到磁盘上的save方法：

In [48]:
frame =pd.read_csv('ex1.csv')

In [49]:
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [50]:
frame.to_pickle('frame_pickle')

In [51]:
!type frame_pickle

�晲      �pandas.core.frame攲	DataFrame敁�)仈}�(�_data攲pandas.core.internals攲BlockManager敁�)仈(]�(�pandas.core.indexes.base攲
_new_Index敁攈�Index敁攠�(�data攲numpy.core.multiarray攲_reconstruct敁攲numpy攲ndarray敁擪 厰Cb攪擱�(KK厰h�dtype敁攲O8擪 K嚁R�(K�|擭NNJ����J����K?t攂塢�(�a攲b攲c攲d攲message攅t攂�name擭u啍R攈
�pandas.core.indexes.range攲
RangeIndex敁攠�(h+N�start擪 �stop擪�step擪u啍R攅]�(hhK 厰h嚁R�(KKK啍h�i8擪 K嚁R�(K�<擭NNJ����J����K t攂塁`              	                     
                                                 攖攂hhK 厰h嚁R�(KKK啍h!塢�(�hello攲world攲foo攅t攂e]�(h
h}�(hhhK 厰h嚁R�(KK厰h!塢�(h%h&h'h(et攂h+Nu啍R攈
h}�(hhhK 厰h嚁R�(KK厰h!塢攈)at攂h+Nu啍R攅}攲0.14.1攠�(�axes攈
�blocks擼�(}�(�values攈:�mgr_locs攲builtins攲slice敁擪 KK嚁R攗}�(hfhEhghjKKK嚁R攗eust攂�_typ攲	dataframe攲	_metadata擼攗b.


In [52]:
pd.read_pickle('frame_pickle')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


## 使用HDF5格式
很多工具都能实现高效读写磁盘上以二进制格式存储的科学数据。HDF5就是其中一个流行的工业级库，它是一个C库，带有许多语言接口，如Java、Python和MATLAB等。HDF5中的HDF指的是层次型数据格式（hierarchical data format）。每个HDF5文件都含有一个文件系统式的节点结构，它使你能够存储多个数据集并支持元数据。与其他简单格式相比，HDF5支持多种压缩器的即时压缩，还能更高效地存储重复模式数据。对于那些非常大的无法直接放入内存的数据集，HDF5就是不错的选择，因为它可以高效地分块读写。
<br />
<br />
Python的HDF5库有两个接口（即PyTables和h5py），它们各自采取了不同的问题解决方式。h5py提供了一种直接而高级的HDF5 API访问接口，而PyTables则抽象了HDF5的许多细节以及提供多种灵活的数据容器、表索引、查询功能以及对核外计算技术（out-of-core computation）的某些支持。

pandas有一个最小化的类似于字典的HDFStore类，它通过PyTables存储pandas对象：

In [53]:
store = pd.HDFStore('mydata.h5')

In [54]:
store['obj1'] = frame

In [55]:
store['obj1_col'] = frame['a']

In [56]:
store

<class 'pandas.io.pytables.HDFStore'>
File path: mydata.h5

In [57]:
store['obj1']

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [58]:
store['obj1_col']

0    1
1    5
2    9
Name: a, dtype: int64

## 使用数据库

In [59]:
import sqlite3

In [60]:
query = """CREATE TABLE test 
           (a VARCHAR(20), b VARCHAR(20),
           c REAL, d INTEGER);"""
con = sqlite3.connect(':memory:')
con.execute(query)
con.commit()

插入几行数据：

In [61]:
data = [('Atlanta', 'Georgia', 1.25, 6),
        ('Tallahassee', 'Florida', 2.6, 3),
        ('Sacramento', 'California', 1.7, 5)]
stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"
con.executemany(stmt, data)
con.commit()

In [62]:
cursor = con.execute('select * from test')

In [63]:
rows = cursor.fetchall()

In [64]:
rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

你可以将这个元组列表传给DataFrame的构造器，但还需要列名（位于游标的description属性中）：

In [65]:
cursor.description

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

In [66]:
DataFrame(rows, columns=list(zip(*cursor.description))[0])

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


pandas有一个可以简化该过程的read_sql函数（位于pandas.io.sql模块），只需传入select语句和连接对象即可：

In [67]:
import pandas.io.sql as sql

In [68]:
sql.read_sql('select * from test', con)

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


## 存取MongoDB中的数据